In [ ]:
from ucimlrepo import fetch_ucirepo

In [ ]:
import pandas as pd

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
wine = fetch_ucirepo(id=109)

In [ ]:
X = wine.data.features

In [ ]:
y = wine.data.targets

In [ ]:
print("\nMissing values:\n", X.isnull().sum())

In [ ]:
X.fillna(X.mean(), inplace=True)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
print("\nFirst 5 rows after normalization:\n", X_scaled.head())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
print("\nTraining set size:", X_train.shape)

In [ ]:
print("Testing set size:", X_test.shape)